# Regressing Transfer Market Value for Football Players 

We can describe this machine learning problem as a regression problem for football players market value prediction. Initially, we have 5 tables which correspond to the following:
- Players Data
- Games Data
- Club Data
- Competition Data
- Appearance Data

Each one of these tables correspond to one another and the main task is obtain one final table (or dataset) with relevant features in order to make market value predictions.
We will try a set of machine learning models after obtaining our dataset and we'll be evaluating our models based on pre-defined regression metrics being Root Mean Squared Error, Mean Squared Error and Mean Absolute Error.

## Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import math

# data preprocessing
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler # Standardizing numerical features
from sklearn.preprocessing import OneHotEncoder # One-hot encoding categorical features

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error

# models
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor)

pd.set_option('display.max_columns', None)

## Exploring All Datasets
During the datasets exploration, we will modify each dataset to account for the final data combination. Each process will be explained thorughout the notebook

### Players Data

In [3]:
dataframe_players = pd.read_csv("dcereijo-player-scores/players")
dataframe_players.head()

,player_id,last_season,current_club_id,name,pretty_name,country_of_birth,country_of_citizenship,date_of_birth,position,sub_position,foot,height_in_cm,market_value_in_gbp,highest_market_value_in_gbp,url
0,214258,2014,3729,andrey-naletov,Andrey Naletov,Russia,Russia,1996-03-31,Defender,Centre-Back,Right,179,NaN,23000.0,https://www.transfermarkt.co.uk/andrey-naletov...
1,45247,2014,1162,damien-perquis,Damien Perquis,France,France,1986-03-08,Goalkeeper,Goalkeeper,Right,186,360000.0,900000.0,https://www.transfermarkt.co.uk/damien-perquis...
2,61389,2014,3729,ivan-lozenkov,Ivan Lozenkov,UdSSR,Russia,1984-04-14,Defender,Centre-Back,Left,182,NaN,270000.0,https://www.transfermarkt.co.uk/ivan-lozenkov/...
3,217036,2014,3729,aleksandr-zakarlyuka,Aleksandr Zakarlyuka,Estonia,Russia,1995-06-24,Midfield,Central Midfield,Right,178,203000.0,203000.0,https://www.transfermarkt.co.uk/aleksandr-zaka...
4,97205,2014,28095,nikita-bezlikhotnov,Nikita Bezlikhotnov,UdSSR,Russia,1990-08-19,Attack,Right Winger,Right,179,225000.0,675000.0,https://www.transfermarkt.co.uk/nikita-bezlikh...


In [4]:
dataframe_players.rename(columns= {"current_club_id": "club_id"}, inplace= True)
dataframe_players.head()

,player_id,last_season,club_id,name,pretty_name,country_of_birth,country_of_citizenship,date_of_birth,position,sub_position,foot,height_in_cm,market_value_in_gbp,highest_market_value_in_gbp,url
0,214258,2014,3729,andrey-naletov,Andrey Naletov,Russia,Russia,1996-03-31,Defender,Centre-Back,Right,179,NaN,23000.0,https://www.transfermarkt.co.uk/andrey-naletov...
1,45247,2014,1162,damien-perquis,Damien Perquis,France,France,1986-03-08,Goalkeeper,Goalkeeper,Right,186,360000.0,900000.0,https://www.transfermarkt.co.uk/damien-perquis...
2,61389,2014,3729,ivan-lozenkov,Ivan Lozenkov,UdSSR,Russia,1984-04-14,Defender,Centre-Back,Left,182,NaN,270000.0,https://www.transfermarkt.co.uk/ivan-lozenkov/...
3,217036,2014,3729,aleksandr-zakarlyuka,Aleksandr Zakarlyuka,Estonia,Russia,1995-06-24,Midfield,Central Midfield,Right,178,203000.0,203000.0,https://www.transfermarkt.co.uk/aleksandr-zaka...
4,97205,2014,28095,nikita-bezlikhotnov,Nikita Bezlikhotnov,UdSSR,Russia,1990-08-19,Attack,Right Winger,Right,179,225000.0,675000.0,https://www.transfermarkt.co.uk/nikita-bezlikh...


In [5]:
dataframe_players.shape

(22604, 15)

### Games Data

In [6]:
dataframe_games = pd.read_csv("dcereijo-player-scores/games")
dataframe_games

,game_id,competition_code,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,stadium,attendance,referee,url
0,2457642,NLSC,2014,Final,2014-08-03,1269,610,1,0,NaN,NaN,Johan Cruijff ArenA,42000.0,Danny Makkelie,https://www.transfermarkt.co.uk/spielbericht/i...
1,2639088,BESC,2013,Final,2014-07-20,58,498,2,1,NaN,NaN,Constant Vanden Stock Stadion,13733.0,Jonathan Lardot,https://www.transfermarkt.co.uk/spielbericht/i...
2,2481145,SUC,2014,final 1st leg,2014-08-19,418,13,1,1,NaN,NaN,Santiago Bernabéu,75897.0,Estrada Fernández,https://www.transfermarkt.co.uk/spielbericht/i...
3,2484338,POSU,2014,Final,2014-08-10,294,2425,3,2,NaN,NaN,Estádio Municipal de Aveiro,30000.0,Duarte Gomes,https://www.transfermarkt.co.uk/spielbericht/i...
4,2502472,FRCH,2014,Final,2014-08-02,583,855,2,0,NaN,NaN,Workers Stadium,39752.0,Clément Turpin,https://www.transfermarkt.co.uk/spielbericht/i...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43955,3643307,CLQ,2021,Qualifying Round 1st leg,2021-08-17,2481,419,3,0,NaN,NaN,Sheriff,5281.0,Cüneyt Cakir,https://www.transfermarkt.co.uk/spielbericht/i...
43956,3629201,ELQ,2021,3rd round 2nd leg,2021-08-12,602,4059,0,1,NaN,NaN,LFF stadionas,NaN,Bartosz Frankowski,https://www.transfermarkt.co.uk/spielbericht/i...
43957,3584561,CLQ,2020,Second Round 2nd leg,2021-07-28,865,371,2,1,NaN,NaN,MCH Arena,4900.0,Bartosz Frankowski,https://www.transfermarkt.co.uk/spielbericht/i...
43958,3643357,ELQ,2021,Qualifying Round 1st leg,2021-08-19,62,255,2,2,NaN,NaN,Sinobo Stadium,14543.0,Felix Zwayer,https://www.transfermarkt.co.uk/spielbericht/i...


### Club Data

In [7]:
dataframe_clubs = pd.read_csv("dcereijo-player-scores/clubs")
dataframe_clubs.head()

,club_id,name,pretty_name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,url
0,3302,ud-almeria,Ud Almeria,ES1,51.66,25,25.7,10,40.0,2,Estadio de los Juegos Mediterráneos,15274,£-4.82m,Francisco Rodríguez,https://www.transfermarkt.co.uk/ud-almeria/sta...
1,5572,niki-volou,Niki Volou,GR1,3.40,24,28.0,6,25.0,1,Panthessaliko Stadio,22700,+-0,Wiljan Vloet,https://www.transfermarkt.co.uk/niki-volou/sta...
2,20698,balikesirspor,Balikesirspor,TR1,1.58,30,24.6,9,30.0,1,Balıkesir Atatürk,15800,+-0,Ismail Ertekin,https://www.transfermarkt.co.uk/balikesirspor/...
3,6894,kayseri-erciyesspor,Kayseri Erciyesspor,TR1,NaN,0,NaN,0,NaN,0,Kayseri Atatürk Spor Kompleksi Yan Açık Saha,2000,+-0,Hikmet Karaman,https://www.transfermarkt.co.uk/kayseri-erciye...
4,1429,cesena-fc,Cesena Fc,IT1,6.82,29,24.9,4,13.8,1,Orogel Stadium - Dino Manuzzi,23860,+£90Th.,Pierpaolo Bisoli,https://www.transfermarkt.co.uk/cesena-fc/star...


In this dataset, we can remove redundant features that we believe can be removed for dimensionality reduction due to the problem formulation:
- pretty_name
- domestic_competition_id
- stadium_name
- stadium_seats
- coach_name 
- url

In [8]:
dataframe_clubs_edited = dataframe_clubs.drop(labels= ["pretty_name", "domestic_competition_id", "stadium_name", "stadium_seats", "coach_name", "url"], axis= 1)

club_columns = [f"club_{x}" if x != "club_id" else x for x in list(dataframe_clubs_edited.columns) ]
mapping_club = {key:value for key, value in zip(list(dataframe_clubs_edited.columns), club_columns)}
dataframe_clubs_edited.rename(columns= mapping_club, inplace= True)
dataframe_clubs_edited

,club_id,club_name,club_total_market_value,club_squad_size,club_average_age,club_foreigners_number,club_foreigners_percentage,club_national_team_players,club_net_transfer_record
0,3302,ud-almeria,51.66,25,25.7,10,40.0,2,£-4.82m
1,5572,niki-volou,3.40,24,28.0,6,25.0,1,+-0
2,20698,balikesirspor,1.58,30,24.6,9,30.0,1,+-0
3,6894,kayseri-erciyesspor,NaN,0,NaN,0,NaN,0,+-0
4,1429,cesena-fc,6.82,29,24.9,4,13.8,1,+£90Th.
...,...,...,...,...,...,...,...,...,...
374,232,spartak-moskau,106.25,25,24.6,8,32.0,11,£-10.82m
375,932,lokomotiv-moskau,71.01,25,24.1,8,32.0,6,£-28.07m
376,7378,portimonense-sc,22.28,30,24.9,26,86.7,4,+£6.41m
377,200,fc-utrecht,31.50,31,24.7,9,29.0,1,+£4.91m


### Competitions Data

In [9]:
dataframe_competitions = pd.read_csv("dcereijo-player-scores/competitions")
dataframe_competitions.head()

,competition_id,name,type,country_id,country_name,domestic_league_code,confederation,url
0,L1,bundesliga,first_tier,40,Germany,L1,europa,https://www.transfermarkt.co.uk/bundesliga/sta...
1,DFB,dfb-pokal,domestic_cup,40,Germany,L1,europa,https://www.transfermarkt.co.uk/dfb-pokal/star...
2,DFL,dfl-supercup,domestic_super_cup,40,Germany,L1,europa,https://www.transfermarkt.co.uk/dfl-supercup/s...
3,NL1,eredivisie,first_tier,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/eredivisie/sta...
4,NLP,toto-knvb-beker,domestic_cup,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/toto-knvb-beke...


### Appearances Data

In [10]:
dataframe_appearances = pd.read_csv("dcereijo-player-scores/appearances")
dataframe_appearances

,player_id,game_id,appearance_id,competition_id,player_club_id,goals,assists,minutes_played,yellow_cards,red_cards
0,52453,2483937.0,2483937_52453,RU1,28095,0,0,90,0,0
1,67064,2479929.0,2479929_67064,RU1,28095,0,0,90,0,0
2,67064,2483937.0,2483937_67064,RU1,28095,0,0,90,0,0
3,67064,2484582.0,2484582_67064,RU1,28095,0,0,55,0,0
4,67064,2485965.0,2485965_67064,RU1,28095,0,0,90,0,0
...,...,...,...,...,...,...,...,...,...,...
1021114,74223,3580335.0,3580335_74223,DK1,678,0,1,90,1,0
1021115,811544,3602561.0,3602561_811544,RU1,2696,0,0,2,0,0
1021116,811544,3602567.0,3602567_811544,RU1,2696,0,0,1,0,0
1021117,811544,3602578.0,3602578_811544,RU1,2696,0,0,1,0,0


After exploring the different datasets we can start by creating the final dataset which will be a combination of all into the players dataset. 
The steps we will be performing are the following:
- [x] Left table merge Games dataset into Appearances
- [x] Drop redundand features after merging
- [x] Merging Players Dataframe with Clubs Dataframe
- [x] Getting Corresponding Club Position
- [x] Aggregating Goals, Assists and Other Corresponding Features
- [x] Specifying Goal - Assist - Minutes Played Per Competition Type
- [x] Merging With Players Dataframe to Obtain the Final Dataframe

## Creating Final Dataset

### 1- Left table merge Games dataset into Appearances

In [11]:
left_df = dataframe_appearances
app_games_merged = left_df.merge(dataframe_games, on= "game_id", how= "left")
app_games_merged= app_games_merged.drop(labels= ["game_id", "url", "referee", "stadium", "attendance", "date", "appearance_id", "season"], axis= 1)
app_games_merged

,player_id,competition_id,player_club_id,goals,assists,minutes_played,yellow_cards,red_cards,competition_code,round,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position
0,52453,RU1,28095,0,0,90,0,0,RU1,2. Matchday,4128.0,28095.0,0.0,1.0,14.0,9.0
1,67064,RU1,28095,0,0,90,0,0,RU1,1. Matchday,2439.0,28095.0,2.0,0.0,6.0,11.0
2,67064,RU1,28095,0,0,90,0,0,RU1,2. Matchday,4128.0,28095.0,0.0,1.0,14.0,9.0
3,67064,RU1,28095,0,0,55,0,0,RU1,3. Matchday,28095.0,121.0,0.0,2.0,10.0,4.0
4,67064,RU1,28095,0,0,90,0,0,RU1,4. Matchday,964.0,28095.0,1.0,0.0,1.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021114,74223,DK1,678,0,1,90,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1021115,811544,RU1,2696,0,0,2,0,0,RU1,20. Matchday,2696.0,3729.0,2.0,2.0,NaN,NaN
1021116,811544,RU1,2696,0,0,1,0,0,RU1,21. Matchday,2696.0,964.0,1.0,1.0,NaN,NaN
1021117,811544,RU1,2696,0,0,1,0,0,RU1,22. Matchday,41231.0,2696.0,2.0,3.0,NaN,NaN


### 2- Drop redundand features after merging

In [12]:
app_games_comp_merged = app_games_merged.merge(dataframe_competitions, on= "competition_id", how= "left")
app_games_comp_merged = app_games_comp_merged.drop(labels= ["confederation", "competition_code", "url", "domestic_league_code", "country_id"], axis= 1)
app_games_comp_merged.rename(columns= {"player_club_id": "club_id"}, inplace= True)
app_games_comp_merged

,player_id,competition_id,club_id,goals,assists,minutes_played,yellow_cards,red_cards,round,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,name,type,country_name
0,52453,RU1,28095,0,0,90,0,0,2. Matchday,4128.0,28095.0,0.0,1.0,14.0,9.0,premier-liga,first_tier,Russia
1,67064,RU1,28095,0,0,90,0,0,1. Matchday,2439.0,28095.0,2.0,0.0,6.0,11.0,premier-liga,first_tier,Russia
2,67064,RU1,28095,0,0,90,0,0,2. Matchday,4128.0,28095.0,0.0,1.0,14.0,9.0,premier-liga,first_tier,Russia
3,67064,RU1,28095,0,0,55,0,0,3. Matchday,28095.0,121.0,0.0,2.0,10.0,4.0,premier-liga,first_tier,Russia
4,67064,RU1,28095,0,0,90,0,0,4. Matchday,964.0,28095.0,1.0,0.0,1.0,12.0,premier-liga,first_tier,Russia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021114,74223,DK1,678,0,1,90,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,superligaen,first_tier,Denmark
1021115,811544,RU1,2696,0,0,2,0,0,20. Matchday,2696.0,3729.0,2.0,2.0,NaN,NaN,premier-liga,first_tier,Russia
1021116,811544,RU1,2696,0,0,1,0,0,21. Matchday,2696.0,964.0,1.0,1.0,NaN,NaN,premier-liga,first_tier,Russia
1021117,811544,RU1,2696,0,0,1,0,0,22. Matchday,41231.0,2696.0,2.0,3.0,NaN,NaN,premier-liga,first_tier,Russia


### 3- Merging Players Dataframe with Clubs Dataframe

In [13]:
dataframe_players_clubs = dataframe_players.merge(dataframe_clubs_edited, on= "club_id", how= "left")
dataframe_players_clubs = dataframe_players_clubs.drop(labels= ["url"], axis= 1)
dataframe_players_clubs

,player_id,last_season,club_id,name,pretty_name,country_of_birth,country_of_citizenship,date_of_birth,position,sub_position,foot,height_in_cm,market_value_in_gbp,highest_market_value_in_gbp,club_name,club_total_market_value,club_squad_size,club_average_age,club_foreigners_number,club_foreigners_percentage,club_national_team_players,club_net_transfer_record
0,214258,2014,3729,andrey-naletov,Andrey Naletov,Russia,Russia,1996-03-31,Defender,Centre-Back,Right,179,NaN,23000.0,arsenal-tula,22.46,25,27.9,7,28.0,5,+£2.21m
1,45247,2014,1162,damien-perquis,Damien Perquis,France,France,1986-03-08,Goalkeeper,Goalkeeper,Right,186,360000.0,900000.0,sm-caen,18.14,25,26.1,9,36.0,2,+£5.85m
2,61389,2014,3729,ivan-lozenkov,Ivan Lozenkov,UdSSR,Russia,1984-04-14,Defender,Centre-Back,Left,182,NaN,270000.0,arsenal-tula,22.46,25,27.9,7,28.0,5,+£2.21m
3,217036,2014,3729,aleksandr-zakarlyuka,Aleksandr Zakarlyuka,Estonia,Russia,1995-06-24,Midfield,Central Midfield,Right,178,203000.0,203000.0,arsenal-tula,22.46,25,27.9,7,28.0,5,+£2.21m
4,97205,2014,28095,nikita-bezlikhotnov,Nikita Bezlikhotnov,UdSSR,Russia,1990-08-19,Attack,Right Winger,Right,179,225000.0,675000.0,fk-ufa,20.30,26,26.8,10,38.5,5,+£419Th.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22599,355370,2021,7775,dylan-saint-louis,Dylan Saint Louis,France,Congo,1995-04-26,Attack,Left Winger,Right,184,1710000.0,1710000.0,hatayspor,27.05,26,26.2,13,50.0,7,+£2.90m
22600,287578,2021,7775,sam-adekugbe,Sam Adekugbe,England,Canada,1995-01-16,Defender,Left-Back,Left,176,1260000.0,1260000.0,hatayspor,27.05,26,26.2,13,50.0,7,+£2.90m
22601,259943,2021,7775,isaac-sackey,Isaac Sackey,Ghana,Ghana,1994-04-04,Midfield,Defensive Midfield,Right,190,810000.0,1130000.0,hatayspor,27.05,26,26.2,13,50.0,7,+£2.90m
22602,354173,2021,2293,adil-demirbag,Adil Demirbag,Turkey,Turkey,1997-12-10,Defender,Centre-Back,Right,185,1080000.0,1080000.0,konyaspor,28.46,25,27.2,13,52.0,11,£-963Th.


### 4-  Getting Corresponding Club Position
In the games dataset, each club has a corresponding position (whether home or away club). In this step, we wish to match the club the player belongs to in the designated game and obtain the corresponding club position. We believe that the position of the club from which the player comes is in fact related to the player market value

In [14]:
def get_club_pos(home_club_id, away_club_id, club_id, df):
    if home_club_id == club_id:
        return df["home_club_position"]
    elif away_club_id == club_id:
        return df["away_club_position"]
    
app_games_comp_merged['club_position'] = app_games_comp_merged.apply(lambda x: get_club_pos(x["home_club_id"], x["away_club_id"], x["club_id"], x), axis=1)
app_games_comp_merged = app_games_comp_merged.drop(["home_club_id", "away_club_id", "home_club_position", "away_club_position", "country_name", "home_club_goals", "away_club_goals"], axis= 1)
app_games_comp_merged

,player_id,competition_id,club_id,goals,assists,minutes_played,yellow_cards,red_cards,round,name,type,club_position
0,52453,RU1,28095,0,0,90,0,0,2. Matchday,premier-liga,first_tier,9.0
1,67064,RU1,28095,0,0,90,0,0,1. Matchday,premier-liga,first_tier,11.0
2,67064,RU1,28095,0,0,90,0,0,2. Matchday,premier-liga,first_tier,9.0
3,67064,RU1,28095,0,0,55,0,0,3. Matchday,premier-liga,first_tier,10.0
4,67064,RU1,28095,0,0,90,0,0,4. Matchday,premier-liga,first_tier,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1021114,74223,DK1,678,0,1,90,1,0,NaN,superligaen,first_tier,NaN
1021115,811544,RU1,2696,0,0,2,0,0,20. Matchday,premier-liga,first_tier,NaN
1021116,811544,RU1,2696,0,0,1,0,0,21. Matchday,premier-liga,first_tier,NaN
1021117,811544,RU1,2696,0,0,1,0,0,22. Matchday,premier-liga,first_tier,NaN


Renaming competition specific features

In [15]:
app_games_comp_merged.rename(columns= {"name": "comp_name", "type": "comp_type"}, inplace= True)
app_games_comp_merged

,player_id,competition_id,club_id,goals,assists,minutes_played,yellow_cards,red_cards,round,comp_name,comp_type,club_position
0,52453,RU1,28095,0,0,90,0,0,2. Matchday,premier-liga,first_tier,9.0
1,67064,RU1,28095,0,0,90,0,0,1. Matchday,premier-liga,first_tier,11.0
2,67064,RU1,28095,0,0,90,0,0,2. Matchday,premier-liga,first_tier,9.0
3,67064,RU1,28095,0,0,55,0,0,3. Matchday,premier-liga,first_tier,10.0
4,67064,RU1,28095,0,0,90,0,0,4. Matchday,premier-liga,first_tier,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1021114,74223,DK1,678,0,1,90,1,0,NaN,superligaen,first_tier,NaN
1021115,811544,RU1,2696,0,0,2,0,0,20. Matchday,premier-liga,first_tier,NaN
1021116,811544,RU1,2696,0,0,1,0,0,21. Matchday,premier-liga,first_tier,NaN
1021117,811544,RU1,2696,0,0,1,0,0,22. Matchday,premier-liga,first_tier,NaN


In [16]:
app_games_comp_merged = app_games_comp_merged.drop(labels= ["club_id"], axis= 1)
app_games_comp_merged

,player_id,competition_id,goals,assists,minutes_played,yellow_cards,red_cards,round,comp_name,comp_type,club_position
0,52453,RU1,0,0,90,0,0,2. Matchday,premier-liga,first_tier,9.0
1,67064,RU1,0,0,90,0,0,1. Matchday,premier-liga,first_tier,11.0
2,67064,RU1,0,0,90,0,0,2. Matchday,premier-liga,first_tier,9.0
3,67064,RU1,0,0,55,0,0,3. Matchday,premier-liga,first_tier,10.0
4,67064,RU1,0,0,90,0,0,4. Matchday,premier-liga,first_tier,12.0
...,...,...,...,...,...,...,...,...,...,...,...
1021114,74223,DK1,0,1,90,1,0,NaN,superligaen,first_tier,NaN
1021115,811544,RU1,0,0,2,0,0,20. Matchday,premier-liga,first_tier,NaN
1021116,811544,RU1,0,0,1,0,0,21. Matchday,premier-liga,first_tier,NaN
1021117,811544,RU1,0,0,1,0,0,22. Matchday,premier-liga,first_tier,NaN


In [17]:
app_games_comp_merged = app_games_comp_merged.drop(labels= ["competition_id", "round", "comp_name"], axis= 1)
app_games_comp_merged

,player_id,goals,assists,minutes_played,yellow_cards,red_cards,comp_type,club_position
0,52453,0,0,90,0,0,first_tier,9.0
1,67064,0,0,90,0,0,first_tier,11.0
2,67064,0,0,90,0,0,first_tier,9.0
3,67064,0,0,55,0,0,first_tier,10.0
4,67064,0,0,90,0,0,first_tier,12.0
...,...,...,...,...,...,...,...,...
1021114,74223,0,1,90,1,0,first_tier,NaN
1021115,811544,0,0,2,0,0,first_tier,NaN
1021116,811544,0,0,1,0,0,first_tier,NaN
1021117,811544,0,0,1,0,0,first_tier,NaN


### 5- Aggregating Goals, Assists and Other Corresponding Features

In [18]:
agg_data = ["goals", "assists", "minutes_played", "yellow_cards", "red_cards"]
aggregation_functions = {key: "sum" if key in agg_data else "first" for key in app_games_comp_merged.columns.tolist()}


app_games_comp_merged_joined = app_games_comp_merged.groupby(app_games_comp_merged['player_id']).aggregate(aggregation_functions)
app_games_comp_merged_joined

,player_id,goals,assists,minutes_played,yellow_cards,red_cards,comp_type,club_position
player_id,,,,,,,,
10,10,24,16,4003,9,0,first_tier,17.0
26,26,0,0,5252,1,0,first_tier,8.0
65,65,13,2,3424,6,1,uefa_champions_league,15.0
80,80,0,0,450,0,0,first_tier,1.0
132,132,2,1,724,2,0,uefa_champions_league,4.0
...,...,...,...,...,...,...,...,...
919173,919173,0,0,3,0,0,first_tier,14.0
926954,926954,0,0,24,1,0,first_tier,10.0
926987,926987,0,0,3,0,0,first_tier,NaN


### 6- Specifying Goal - Assist - Minutes Played Per Competition Type

In [19]:
comp_type_list= app_games_comp_merged["comp_type"].unique().tolist()
comp_type_list

['first_tier',
 'domestic_cup',
 'uefa_champions_league',
 'europa_league',
 'europa_league_qualifying',
 'uefa_champions_league_qualifying',
 'domestic_super_cup',
 'fifa_club_world_cup',
 'uefa_super_cup',
 'uefa_europa_conference_league_qualifiers']

**Get Data We want to dissern by competition type**

In [20]:
agg_data = ["goals", "assists", "minutes_played"]

In [ ]:
for data in agg_data:
    for comp_type in comp_type_list:
        app_games_comp_merged_joined[f"{data}_{comp_type}"] = app_games_comp_merged.groupby(["player_id"]).apply(lambda x: x.loc[x["comp_type"] == comp_type, data].sum())

In [ ]:
app_games_comp_merged_joined = app_games_comp_merged_joined.drop(labels= ["comp_type", "player_id"], axis= 1)
app_games_comp_merged_joined = app_games_comp_merged_joined.rename({"goals": "total_goals", "assists":"total_assists", "minutes_played": "total_minutes_played"})
app_games_comp_merged_joined

### 7- Merging With Players Dataframe

In [ ]:
final_df = app_games_comp_merged_joined.merge(dataframe_players_clubs, on= "player_id", how= "left")
final_df

In [ ]:
final_df = final_df[final_df["position"] == "Attack"].drop(labels= ["position","club_id", "pretty_name"], axis= 1)
final_df = final_df.reset_index(drop= True)
final_df

In [ ]:
final_df[final_df.duplicated(["player_id"])]

## Data Pre-Processing
**Data Cleaning Tasks**:
- [x] Convert *club_net_transfer_record* feature to numerical
- [x] Convert *date_of_birth* feature to numerical age
- [x] Split into Train/Test partitions
- [x] Fill 0 values in *height* with mean
- [x] Fill all NaN values with mean
- [x] Converting club_position into INTEGER Categorical Feature
- [x] Converting last_season into INTEGER Categorical Feature

### Converting club_net_transfer_record Feature to Numerical

In [ ]:
def convert_transfer_to_num(club_transfer_rec):
    
    if isinstance(club_transfer_rec, int):
        return club_transfer_rec
    else:
        th = 0
        symb_idx = 0
        first_num_idx = 2

        res = 0

        if club_transfer_rec == None or club_transfer_rec == "":
            return None
        if club_transfer_rec[-1] == "0" and club_transfer_rec[:2] == "+-":
            return 0    

        if club_transfer_rec[-1] == "m":
            th = 1_000_000
            symb_idx = club_transfer_rec.index("m")
        elif "Th." in club_transfer_rec:
            th = 1_000
            symb_idx = club_transfer_rec.index("Th.")

        try:
            res = int(th * float(club_transfer_rec[first_num_idx:symb_idx]))
        except ValueError:
            th = 1
            res = int(th* float(club_transfer_rec[first_num_idx:]))

        if club_transfer_rec[0] == "+" and club_transfer_rec[1] == "£":
            return res
        elif club_transfer_rec[0] == "£" and club_transfer_rec[1] == "-":
            return res*-1
        else:
            return None

In [ ]:
final_df["club_net_transfer_record"] = final_df["club_net_transfer_record"].apply(lambda x: convert_transfer_to_num(x))
final_df

### Converting *date_of_birth* Feature to Numerical Age

In [ ]:
import dateutil
import datetime

def convert_dob_to_age(dob):
    if dob == None or not(pd.notna(dob)):
        return None
    # Get the current date
    now = datetime.datetime.utcnow()
    now = now.date()
    
    # Convert isoformat of date to datetime object
    date = datetime.datetime.fromisoformat(dob)
    # Get the difference between the current date and the birthday
    age = dateutil.relativedelta.relativedelta(now, date)
    age = age.years

    return age

In [ ]:
final_df["date_of_birth"] = final_df["date_of_birth"].apply(lambda x: convert_dob_to_age(x))
final_df.rename(columns= {"date_of_birth": "age"}, inplace= True)

In [ ]:
final_df

### Splitting Data To Train/Test 85%/15%
We split the dataset into train and test before filling out NaN or any missing values in order to have corresponding distributions in the train and test sets. Filling NaN or missing values before splitting the dataset might introduce data leaking into the test set where the machine learning can possibly perform better but that will not be the case when we deploy the model in the real world since the model had leaked data from the training set.

In [ ]:
SEED = 11

X_train, X_test, y_train, y_test = train_test_split(final_df.drop(labels= ["market_value_in_gbp"], axis= 1),
                                                   final_df["market_value_in_gbp"],
                                                   test_size=0.15,
                                                   random_state=SEED) 

In [ ]:
print(f"X_train shape: {X_train.shape}, y_train_shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test_shape: {y_test.shape}")

### Fill 0 Values in Height with Mean

In [ ]:
zero_height_train, zero_height_test = X_train[X_train["height_in_cm"]== 0], X_test[X_test["height_in_cm"]== 0]
print(f"We have {zero_height_train.shape[0]} datapoints with zero values for the height")

height_mean_train, height_mean_test = 0.0, 0.0
height_total_train, height_total_test = 0.0, 0.0

for height_train, height_test in zip(X_train["height_in_cm"], X_test["height_in_cm"]):
    if height_train != 0:
        height_total_train += height_train
    if height_test != 0:
        height_total_test += height_test
        
height_mean_train = height_total_train / (X_train["height_in_cm"].shape[0] - zero_height_train.shape[0])
height_mean_test = height_total_test / (X_test["height_in_cm"].shape[0] - zero_height_test.shape[0])
print(f"Height mean without zero values: train: {height_mean_train:.3f}, test: {height_test:.3f}")

In [ ]:
def fill_zeros_mean(height, height_mean):
    if height == 0:
        return height_mean
    else:
        return height

In [ ]:
X_train["height_in_cm"] = X_train["height_in_cm"].apply(lambda x: fill_zeros_mean(x, height_mean_train))
X_test["height_in_cm"] = X_test["height_in_cm"].apply(lambda x: fill_zeros_mean(x, height_mean_test))
X_train

In [ ]:
zero_height_train, zero_height_test = X_train[X_train["height_in_cm"]== 0], X_test[X_test["height_in_cm"]== 0] 
print(f"We have {zero_height_train.shape[0]}, {zero_height_test.shape[0]} (train, test) datapoints with zero values for the height")

### Filling NaN Values

In [ ]:
def summarize_missingness(df, label= None):
    '''
    Utility function to summarize missing or NaN values
    '''
    nulls = df.isnull()
    counts = nulls.sum()
    percs = nulls.mean().mul(100.)
    
    if label is None:
        label = ""
    nulls_df = pd.DataFrame({f'{label} Count of missing/NaN values': counts, 'Percentage of missing values': percs}, 
                            index=counts.index)
    
    display(nulls_df)

In [ ]:
vars_with_na = [col for col in X_train.columns if X_train[col].isnull().sum() > 0]
data_label = ["X_train", "X_test"]
for i, dataframe in enumerate([X_train, X_test]):
    summarize_missingness(dataframe[vars_with_na], label= data_label[i])

In [ ]:
vars_with_na = [col for col in X_train if X_train[col].isnull().sum() > 0]
cat_vars_with_na = ['club_name', 'country_of_birth', 'foot']
print("Categorical Features with NaNs:", cat_vars_with_na)
num_vars_with_na = list(set(vars_with_na) - set(cat_vars_with_na))
print("Numerical Features with NaNs:", num_vars_with_na)

#### Filling Numerical Features with NaNs

In [ ]:
X_train[num_vars_with_na] = X_train[num_vars_with_na].fillna((X_train[num_vars_with_na].mean()))
X_test[num_vars_with_na] = X_test[num_vars_with_na].fillna((X_test[num_vars_with_na].mean()))

#### Filling Categorical Features with NaNs

In [ ]:
for cat_var in cat_vars_with_na:
    X_train[cat_var] = X_train[cat_var].fillna(X_train[cat_var].value_counts().index[0])
    X_test[cat_var] = X_test[cat_var].fillna(X_test[cat_var].value_counts().index[0])

In [ ]:
vars_with_na = [col for col in X_train.columns if X_train[col].isnull().sum() > 0]
data_label = ["X_train", "X_test"]
for i, dataframe in enumerate([X_train, X_test]):
    summarize_missingness(dataframe[vars_with_na], label= data_label[i])

#### Filling y_train and y_test NaN values

In [ ]:
print(f"We have {y_train.isnull().sum()} NaN values in y_train and {y_test.isnull().sum()} NaN values in y_test")

We fill *y_train* and *y_test* values with the mean:

In [ ]:
y_train = y_train.fillna(y_train.mean())
y_test = y_test.fillna(y_test.mean())
print(f"We have {y_train.isnull().sum()} NaN values in y_train and {y_test.isnull().sum()} NaN values in y_test")

### Converting club_position into INTEGER Categorical Feature
After filling missing values of the *club_position* feature, the obtained results show floating point values (since we filled the NaNs with the mean club_position). At this point we can treat this feature as a categorical feature with integer values, therefore we use **round** function to round the position to the nearest integer. 

In [ ]:
X_train["club_position"], X_test["club_position"] = X_train["club_position"].apply(lambda x: int(round(x))), X_test["club_position"].apply(lambda x: int(round(x)))

### Converting last_season into INTEGER Categorical Feature
Similarly, we convert data in the last_season feature to integers in order to consider this feature as categorical.

In [ ]:
X_train["last_season"], X_test["last_season"] = X_train["last_season"].apply(lambda x: int(round(x))), X_test["last_season"].apply(lambda x: int(round(x)))

## EDA
In this section, we will aim to explore the data, try to make sense of the correlation between the features, their influce on the target variable and their corresponding distributions. We will complete the following steps:

- [x] Main Data Info
- [x] Representing Unique Categorical Values
- [x] Target Variable Distribution
- [x] Categorical Variables Distributions
- [x] Influence Categorical Features on Player Market Value
- [x] Player Specific Numerical Features Distributions
- [x] Club Specific Numerical Features Distributions
- [x] Influence of Some Numerical Features on the Target Variable
- [x] Calculating Correlation

### Main Data Info

In [ ]:
print(f"X_train shape: {X_train.shape}\nX_test shape: {X_test.shape}")
X_train.head()

In the final version of the dataset, we have 4 459 samples in the training set and 788 samples in the testing set. In this dataset we have 55 features after combining the 5 dataframes shown above. We cleaned and pre-processed the data in a way where irrelevant columns were removed such as *URL*, *PRETTY-NAME*, *COMPETITION-ID* and others. 
In the first steps, we wish to model our problem as a regression problem where we wish to predict player market value. However, we will try to explore the possibility of modeling this problem as classification problem where we divide market values into brakets and players will be classified according to each braket containing a range of the market value (e.g braket_1: market value range [0-540_000]. 
But first, we wish to understand our data better and see the distributions of the features in the dataset as well as correlation with the target variable and many other aspects.

In [ ]:
X_train.describe()

In [ ]:
X_test.describe()

### Representing Unique Categorical Values
**Getting the Categorical Features Without the names**

In [ ]:
categorical_features_wo_names = ["country_of_birth", "country_of_citizenship", "sub_position", "foot", "club_position", "last_season"]
categorical_features_wo_names

In [ ]:
# get categorical data having multiple labels
categorical_df = pd.DataFrame()
for categorical_feature in categorical_features_wo_names:
    tmp_df = pd.DataFrame({categorical_feature:dataframe[categorical_feature].unique()})
    categorical_df = pd.concat([categorical_df, tmp_df], axis=1)
categorical_df

### Target Variable Distribution
The target variable is the market value of a player in gpb. However, we rescale the target variable in both the training and testing set by a factor of 1000. Therefore, this variable is represented as market value (1000) in gpb. We perform this operation to make visualization easier

In [ ]:
y_train, y_test = y_train/1000, y_test/1000

In [ ]:
# Visualizing the target variable
plt.figure(figsize=(12,8))
sns.distplot(y_train, label=f'train target, skew: {y_train.skew():.2f}')
sns.distplot(y_test, label=f'test target, skew: {y_test.skew():.2f}')
plt.legend(loc='best')
plt.show()

The target variable is positively skewed, we can perform a log transformation to render the target distribution more gaussian like to **avoid biasing our ML model** to the majority labels.

In [ ]:
# log transform target
y_train = np.log(y_train)
y_test = np.log(y_test)

# Visualizing the target variable
plt.figure(figsize=(12,8))
sns.distplot(y_train, label=f'train target, skew: {y_train.skew():.2f}')
sns.distplot(y_test, label=f'test target, skew: {y_test.skew():.2f}')
plt.legend(loc='best')
plt.show()

We perform the log transformation on the target variable for the following reasons:
1. Log transform guarantees all values are POSITIVE. In many real-world appications the labels are naturally positive.
2. Log transform makes large numbers even larger, pulling the distribution to the right to bigger values.
3. Alternatively, the log transform makes smaller numbers even smaller, stretching left values even farther away by making smaller. Consequently, the entire distribution gets centered and transformed into a normalized distribution

### Categorical Variables Distributions
We recall the categorical features without the names:

In [ ]:
categorical_features_wo_names

We also remove the "country_of_birth" as well as "country_of_citizenship" from this list for visualization purposes. These features will be visualized later.

In [ ]:
categorical_features_rectified = categorical_features_wo_names[2:]
categorical_features_rectified

In [ ]:
for cat_var in categorical_features_rectified:
    fig, ax = plt.subplots(1, 2, figsize=(12,5))
    try:
        sns.countplot(y=cat_var, data=X_train, label='train', ax=ax[0])
        sns.countplot(y=cat_var, data=X_test, label='test', ax=ax[1])
        ax[0].set_title(cat_var + " Train")
        ax[1].set_title(cat_var + " Test")
        plt.show()
    except Exception as e:
        print(e)

For the "country_of_birth" as well as "country_of_citizenship" we perform the following visualization

In [ ]:
cob_coc_categorical_features = categorical_features_wo_names[:2]
cob_coc_categorical_features

In [ ]:
for col in cob_coc_categorical_features:
    fig, ax = plt.subplots(1, 2, figsize=(12,25))
    sns.countplot(y= col, data= X_train, label= "train", ax=ax[0])
    ax[0].set_title(f"{col} train")
    sns.countplot(y= col, data= X_test,label= "test", ax=ax[1])
    ax[1].set_title(f"{col} test")        
    plt.show()

### Influence Categorical Features on Player Market Value
Recalling categorical features:

In [ ]:
categorical_features_rectified

We note in the following bar plot: 
- the lines signify the confidence interval

In [ ]:
for col in categorical_features_rectified:
    try:
        fig, ax = plt.subplots(1,2, figsize=(12,5))
        sns.barplot(x=X_train[col], y=np.exp(y_train), estimator=np.median, ax=ax[0])
        ax[0].set_title(f"market_value_in_gbp median verus {col} train")
        sns.barplot(x=X_test[col], y=np.exp(y_test), estimator=np.median, ax=ax[1])
        ax[1].set_title(f"market_value_in_gbp median verus {col} train")
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(e)

### Player Specific Numerical Features Distributions

In [ ]:
# Player Properties variables
player_prop_cols = ["age", "height_in_cm", "highest_market_value_in_gbp", "last_season", "goals", "assists", "minutes_played", "yellow_cards", "red_cards"]

for col in player_prop_cols:
    fig, ax = plt.subplots(1, 2, figsize=(12,5))
    sns.distplot(X_train[col], ax=ax[0], color= "blue")
    ax[0].set_title(f"{col} train")
    sns.distplot(X_test[col], ax=ax[1], color= "blue")
    ax[1].set_title(f"{col} test")        
    plt.show()

In [ ]:
g = sns.jointplot("goals", "minutes_played", data=X_train, kind="reg", color = 'green', size = 10)

### Club Specific Numerical Features Distributions

In [ ]:
# club Properties variables
club_prop_cols = ["club_total_market_value", "club_squad_size", "club_average_age", "club_foreigners_number", "club_foreigners_percentage", "club_national_team_players", "club_net_transfer_record"]

for col in club_prop_cols:
    fig, ax = plt.subplots(1, 2, figsize=(12,5))
    sns.distplot(X_train[col], ax=ax[0], color= "green")
    ax[0].set_title(f"{col} train")
    sns.distplot(X_test[col], ax=ax[1], color= "green")
    ax[1].set_title(f"{col} test")        
    plt.show()

### Influence of Some Numerical Features on the Target Variable

In [ ]:
num_cols = club_prop_cols + player_prop_cols
num_cols

In [ ]:
for col in num_cols:
    fig, ax = plt.subplots(1, 2, figsize=(12,5))
    sns.scatterplot(x=X_train[col], y=np.exp(y_train), ax=ax[0])
    ax[0].set_title(col + " train")
    sns.scatterplot(x=X_test[col], y=np.exp(y_test), ax=ax[1])
    ax[1].set_title(col + " test")        
    plt.show()

### Calculating Correlation

In [ ]:
cor = final_df.drop(labels= ["player_id"], axis= 1).corr()
cor.head()

In [ ]:
cor_target = abs(cor["market_value_in_gbp"])
cor_target.sort_values()

From the correleation analysis we can see which features are the 5 most correlated to the target variable:
- highest_market_value_in_gbp
- club_total_market_value
- minutes_played_uefa_champions_league
- goals_uefa_champions_league
- goals

## Feature Engineering
In this final section before model training, we will engineer some of the features in the training dataset like goals/assists and the minutes played. In addition, in this section we will normalize numerical variables and one-hot-encode categorical variables. The full steps conducted in this section are shown below:

- [x] Create a goals per minute feature
- [x] Create an assists per minute feature
- [x] Drop goals and assists features
- [x] Normalize numerical features 
- [x] One-Hot-Encode categorical features

### Creating Goals Per Minute Feature

In [ ]:
X_train["goals_per_minute"], X_test["goals_per_minute"] = X_train["goals"]/X_train["minutes_played"], X_test["goals"]/X_test["minutes_played"]

In [ ]:
X_train, X_test = X_train.drop(labels= ["goals"], axis= 1), X_test.drop(labels= ["goals"], axis= 1)

In [ ]:
X_train.head()

### Creating Assists Per minute Feature

In [ ]:
X_train["assists_per_minute"], X_test["assists_per_minute"] = X_train["assists"]/X_train["minutes_played"], X_test["assists"]/X_test["minutes_played"]

In [ ]:
X_train, X_test = X_train.drop(labels= ["assists"], axis= 1), X_test.drop(labels= ["assists"], axis= 1)

In [ ]:
X_train.head()

### Normalize Numerical Features and One-Hot-Encode Categorical Features

In [ ]:
X_train, X_test = X_train.drop(labels= ["player_id", "name", "country_of_birth"], axis= 1), X_test.drop(labels= ["player_id"], axis= 1)

In [ ]:
all_vars = X_train.columns.to_list()

cat_vars = ['club_position', 'last_season', 'country_of_citizenship', 'sub_position', 'foot', 'club_name']
num_vars = list(set(all_vars) - set(cat_vars))

In [ ]:
# Select & separate the categorical & numerical columns in the training data
X_train_cat = X_train[[c for c in X_train.columns if c in cat_vars]]
X_train_num = X_train[[c for c in X_train.columns if c in num_vars]]

#  Select & separate the categorical & numerical columns in the testing data
X_test_cat = X_test[[c for c in X_test.columns if c in cat_vars]]
X_test_num = X_test[[c for c in X_test.columns if c in num_vars]]

In [ ]:
# Standardizing numerical feature columns
scaler = StandardScaler()
X_train_num_1 = scaler.fit_transform(X_train_num) 

X_test_num_1 = scaler.transform(X_test_num)

In [ ]:
# One-hot encoding categorical feature columns
ohe = OneHotEncoder(sparse=False,handle_unknown='ignore')
X_train_cat_1 = ohe.fit_transform(X_train_cat) 

X_test_cat_1 = ohe.transform(X_test_cat)

In [ ]:
# Put all features back together into a total training and testing sets
X_train_new = np.concatenate((X_train_cat_1, X_train_num_1) , axis=1)
X_test_new = np.concatenate((X_test_cat_1, X_test_num_1) , axis=1)

In [ ]:
print(f"Final X_train/X_test <X_train_new>/<X_test_new> have {X_train_new.shape[1]} features")
print(f"X_train_new has shape: {X_train_new.shape}\nX_test_new has shape: {X_test_new.shape}")

## Regression Model Training
In this part, we will attempt to train a machine learning model to predict the target variable market_value_in_gbp. The following models will be trained: 

- [x] LinearRegression
- [x] SVR
- [x] RandomForestRegressor
- [x] GradientBoostingRegressor

In addition we will monitor the following metrics to evaluate our models:
- MSE
- RMSE
- MEAN (Mean Absolute Error)

In [ ]:
# Train & compare a collection of regressor models
regressor_1 = LinearRegression()
regressor_2 = SVR()
regressor_3 = RandomForestRegressor(random_state= SEED, n_estimators=10)
regressor_4 = GradientBoostingRegressor(random_state= SEED, n_estimators=10)

# Select type of machine learning model
models = []
models.append(regressor_1)
models.append(regressor_2)
models.append(regressor_3)
models.append(regressor_4)

# Train & evaluate each model
metrics = {"MSE": mean_squared_error, "RMSE": mean_squared_error, "MAN": mean_absolute_error}
scores = {"MSE": [], "RMSE": [], "MAN": []}
names = ['LR', 'SVR', 'RF', 'GB']

fig, axes = plt.subplots(nrows= 2, ncols= 2, figsize= (10, 5))
fig.suptitle("Model Comparison on Different Metrics", fontsize=18, y=0.95)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.delaxes(axes[1][1])

for metric, ax in zip(metrics.keys(), axes.ravel()):    
    for i, model in enumerate(models):
        model.fit(X= X_train_new, y= y_train)

        predicted_output = model.predict(X= X_test_new)
        
        if metric == "RMSE":
            score = metrics[metric](y_test, predicted_output, squared= False)
        else:
            score = metrics[metric](y_test, predicted_output)
        
        scores[metric].append(score)
        output = f"Metric: {metric}, Model: {names[i]}: {score:.3f}"
        print(output)
    
    ax.barh(names[1:len(names)], scores[metric][1:len(scores[metric])], align='center', alpha=0.5) # skip LR score
    ax.set_xlabel(metric)
    ax.set_ylabel('Model')
    ax.set_title(f"Model Comparison - {metric}")

fig.subplots_adjust(hspace=1)
plt.show()

Comparing these regression models against the defined metrics, we can choose the RandomForest regression for this specific task since it calculated the lowest metric values accross all the metrics.